<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=bbd30b5f699fb4df53172c0e3b1e01a00fc7dff7c37f378f122f8d5c9421a04b
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-16 15:16:14
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: -62.47 K (-0.44%)
Current PnL: -26.62 L (-17.2%)
CY Booked + Current PnL: -12.33 L (-7.97%)
-------------------
Total profit:  1.38 L
Total loss:  -28.00 L
-------------------
Total Booked + Current PnL: 14.65 L (11.46%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.18%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.26 L (64.98%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.24%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.99,-18.11,22.22,0.09,18338.0,-18247.0,82530.0,91.46,35.0,M-SC,2.33,253.0,-1.00,0.59,8.43,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.27,8.94,9.87,19.69,20617.0,17148.0,208890.0,-1.63,58.0,X-MC,2.53,78.0,0.83,1.49,16.35,XY25,NTT,AC
39,INDIGOPNTS,1408.00,-1.87,-13.32,15.39,0.02,23286.0,-23254.0,151305.0,130.47,49.0,M-SC,14.79,234.0,-1.00,1.08,31.61,OX40N,NTT,PAINTS
50,MASFIN,398.61,-2.85,-2.88,25.67,22.05,24428.0,-2820.0,95160.0,-16.44,52.0,H-SC,7.21,164.0,-0.12,0.68,38.32,XR,ATH,FINANCE
43,ITC,452.00,-0.16,-1.56,12.54,10.78,29666.0,-3746.0,236572.0,-41.56,45.0,X-LC,1.96,5.0,-0.13,1.69,3.60,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJOOENG,143.10,6.87,-33.82,110.97,39.61,75271.0,-34670.0,67830.0,-55.26,38.0,H-SC,28.03,135.0,-0.46,0.49,6.87,AR,ATH,MISC
16,CERA,9475.00,3.24,-30.08,77.19,23.89,94938.0,-52911.0,122992.0,-32.46,40.0,H-SC,10.57,157.0,-0.56,0.88,6.22,OX40N,NTT,CERAMICS
25,FINCABLES,1641.55,2.77,-1.30,110.66,107.91,161254.0,-1924.0,145720.0,-19.74,60.0,M-SC,12.52,216.0,-0.01,1.04,8.65,OX40N,ATH,CABLES
6,ASIANTILES,137.00,2.24,-16.66,117.98,81.67,92243.0,-15625.0,78185.0,7027.78,46.0,L-SC,12.38,271.0,-0.17,0.56,51.41,XR,NTT,CERAMICS
44,JCHAC,2282.00,1.63,-33.88,51.23,-0.01,43291.0,-43301.0,84504.0,17182.61,39.0,M-SC,7.32,233.0,-1.00,0.60,8.16,OX40N,NTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,-4.08,-40.82,128.57,35.27,86399.0,-46349.0,67200.0,-53.72,47.0,L-SC,10.75,270.0,-0.54,0.48,64.47,XR,NTT,HOTELS
50,MASFIN,398.61,-2.85,-2.88,25.67,22.05,24428.0,-2820.0,95160.0,-16.44,52.0,H-SC,7.21,164.0,-0.12,0.68,38.32,XR,ATH,FINANCE
63,SFL,1287.00,-2.36,-44.32,127.43,26.63,186050.0,-116216.0,146002.0,10.55,25.0,M-SC,11.68,239.0,-0.62,1.04,0.00,XY24,NTT,MISC
21,DEN,75.00,-2.25,-40.77,139.85,42.07,69970.0,-34432.0,50032.0,78.76,46.0,M-SC,7.81,235.0,-0.49,0.36,6.25,AR,NTT,ENTERTAINMENT
13,BSOFT,831.70,-2.04,-22.79,95.28,50.79,103478.0,-32049.0,108604.0,-1.04,62.0,H-SC,6.32,151.0,-0.31,0.78,26.45,XR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,2.77,-1.30,110.66,107.91,161254.0,-1924.0,145720.0,-19.74,60.0,M-SC,12.52,216.0,-0.01,1.04,8.65,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.87,-13.32,15.39,0.02,23286.0,-23254.0,151305.0,130.47,49.0,M-SC,14.79,234.0,-1.00,1.08,31.61,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.99,-18.11,22.22,0.09,18338.0,-18247.0,82530.0,91.46,35.0,M-SC,2.33,253.0,-1.00,0.59,8.43,OX40N,NTT,DURABLES
66,SIS,528.00,0.05,-24.19,59.95,21.26,50464.0,-26856.0,84176.0,1986.01,50.0,H-SC,5.12,166.0,-0.53,0.60,13.73,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.87,-25.11,51.52,13.47,104045.0,-67716.0,201951.0,-69.28,35.0,H-SC,5.21,158.0,-0.65,1.44,2.69,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.98,6.3,62.05,72.27,111771.0,10682.0,180130.0,-8.8,63.0,M-LC,2.45,99.0,0.1,1.29,12.87,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.98,6.30,62.05,72.27,111771.0,10682.0,180130.0,-8.80,63.0,M-LC,2.45,99.0,0.10,1.29,12.87,XR,NTT,IT
25,FINCABLES,1641.55,2.77,-1.30,110.66,107.91,161254.0,-1924.0,145720.0,-19.74,60.0,M-SC,12.52,216.0,-0.01,1.04,8.65,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.94,-4.49,116.44,106.72,137164.0,-5538.0,117798.0,-52.73,34.0,H-SC,3.01,139.0,-0.04,0.84,17.84,AR,ATH,MISC
1,ABB,7934.00,-0.68,-3.70,51.15,45.57,128873.0,-9667.0,251952.0,-38.07,57.0,H-MC,4.55,120.0,-0.08,1.80,7.21,AR,NTT,ELECTRICAL
37,IEX,219.00,-1.26,-4.64,55.85,48.63,106498.0,-9268.0,190686.0,-36.48,46.0,H-SC,14.06,136.0,-0.09,1.36,6.38,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.35,-18.24,90.15,55.47,179325.0,-44368.0,198918.0,-27.60,25.0,X-MC,12.93,64.0,-0.25,1.42,1.92,X40N,ATH,FINANCE
53,PAGEIND,51605.08,-1.03,-9.74,41.93,28.11,76229.0,-19610.0,181800.0,-34.62,28.0,X-MC,12.94,55.0,-0.26,1.30,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-0.46,-29.18,73.64,22.98,117817.0,-65908.0,159990.0,-26.74,29.0,X-LC,6.80,3.0,-0.56,1.14,0.63,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-1.06,-38.77,120.83,35.20,95189.0,-49891.0,78779.0,3.25,31.0,X-SC,16.12,92.0,-0.52,0.56,0.07,X40,NTT,FOOTWEAR
8,AWL,485.00,-1.20,-22.97,99.26,53.50,230727.0,-69304.0,232447.0,-62.74,31.0,X-MC,2.80,58.0,-0.30,1.66,3.24,XY24,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.03,-1.07,25.83,24.48,64355.0,-2706.0,249150.0,-81.09,44.0,X-SC,0.33,86.0,-0.04,1.78,21.82,X40N,NTT,MISC
43,ITC,452.0,-0.16,-1.56,12.54,10.78,29666.0,-3746.0,236572.0,-41.56,45.0,X-LC,1.96,5.0,-0.13,1.69,3.60,X40,NTT,FMCG
33,HINDUNILVR,2922.0,-0.50,-11.22,28.04,13.67,63990.0,-28853.0,228210.0,-17.95,32.0,X-LC,2.14,14.0,-0.45,1.63,7.74,XY25,NTT,FMCG
36,ICICIPRULI,790.0,-1.61,5.58,23.81,30.73,49526.0,10999.0,208004.0,-19.08,58.0,X-MC,2.24,75.0,0.22,1.49,19.06,X40,ATH,INSURANCE
78,UNITDSPR,1644.0,0.73,6.74,13.15,20.77,32864.0,15772.0,249916.0,-2.63,58.0,X-MC,2.46,62.0,0.48,1.79,14.12,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.34,-25.55,120.48,64.15,213442.0,-60791.0,177160.0,-56.31,34.0,X-SC,2.60,82.0,-0.28,1.27,0.00,XY24,BTT,CEMENT
53,PAGEIND,51605.08,-1.03,-9.74,41.93,28.11,76229.0,-19610.0,181800.0,-34.62,28.0,X-MC,12.94,55.0,-0.26,1.30,0.00,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-1.06,-38.77,120.83,35.20,95189.0,-49891.0,78779.0,3.25,31.0,X-SC,16.12,92.0,-0.52,0.56,0.07,X40,NTT,FOOTWEAR
75,TMPV,600.00,-0.46,-29.18,73.64,22.98,117817.0,-65908.0,159990.0,-26.74,29.0,X-LC,6.80,3.0,-0.56,1.14,0.63,XY24,NTT,AUTO
59,RELAXO,1176.00,-0.85,-48.09,193.41,52.31,145734.0,-69810.0,75350.0,-44.73,40.0,X-SC,6.95,91.0,-0.48,0.54,0.86,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.36,-29.25,101.89,42.84,46863.0,-19012.0,45994.0,-53.72,44.0,X-SC,35.52,83.0,-0.41,0.33,4.06,XY24,NTT,MISC
59,RELAXO,1176.00,-0.85,-48.09,193.41,52.31,145734.0,-69810.0,75350.0,-44.73,40.0,X-SC,6.95,91.0,-0.48,0.54,0.86,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.06,-38.77,120.83,35.20,95189.0,-49891.0,78779.0,3.25,31.0,X-SC,16.12,92.0,-0.52,0.56,0.07,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.81,-4.90,30.48,24.08,36796.0,-6227.0,120723.0,-12.55,32.0,X-SC,5.12,89.0,-0.17,0.86,13.75,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.21,-16.51,71.66,43.32,97443.0,-26892.0,135980.0,-28.31,37.0,X-SC,4.74,90.0,-0.28,0.97,4.41,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.81,-11.91,37.02,20.71,123353.0,-45031.0,333206.0,-24.27,59.0,X-LC,4.46,1.0,-0.37,2.38,11.34,X40,ATH,IT
41,INFY,1972.00,-0.93,8.36,23.88,34.24,80966.0,26165.0,339053.0,-15.65,59.0,X-LC,4.00,2.0,0.32,2.43,17.34,X40,NTT,IT
75,TMPV,600.00,-0.46,-29.18,73.64,22.98,117817.0,-65908.0,159990.0,-26.74,29.0,X-LC,6.80,3.0,-0.56,1.14,0.63,XY24,NTT,AUTO
81,VBL,671.64,0.05,-3.47,40.55,35.67,123624.0,-10974.0,304868.0,-14.99,56.0,X-LC,4.13,4.0,-0.09,2.18,9.90,X40N,ATH,FMCG
43,ITC,452.00,-0.16,-1.56,12.54,10.78,29666.0,-3746.0,236572.0,-41.56,45.0,X-LC,1.96,5.0,-0.13,1.69,3.60,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.76,-37.63,113.07,32.89,54448.0,-29052.0,48154.0,-694.37,54.0,L-MC,5.96,259.0,-0.53,0.34,32.61,XR,NTT,BANKS
6,ASIANTILES,137.00,2.24,-16.66,117.98,81.67,92243.0,-15625.0,78185.0,7027.78,46.0,L-SC,12.38,271.0,-0.17,0.56,51.41,XR,NTT,CERAMICS
50,MASFIN,398.61,-2.85,-2.88,25.67,22.05,24428.0,-2820.0,95160.0,-16.44,52.0,H-SC,7.21,164.0,-0.12,0.68,38.32,XR,ATH,FINANCE
13,BSOFT,831.70,-2.04,-22.79,95.28,50.79,103478.0,-32049.0,108604.0,-1.04,62.0,H-SC,6.32,151.0,-0.31,0.78,26.45,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.87,-13.32,15.39,0.02,23286.0,-23254.0,151305.0,130.47,49.0,M-SC,14.79,234.0,-1.00,1.08,31.61,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.7,-2.04,-22.79,95.28,50.79,103478.0,-32049.0,108604.0,-1.04,62.0,H-SC,6.32,151.0,-0.31,0.78,26.45,XR,ATH,IT
84,WIPRO,420.0,-0.98,6.30,62.05,72.27,111771.0,10682.0,180130.0,-8.80,63.0,M-LC,2.45,99.0,0.10,1.29,12.87,XR,NTT,IT
41,INFY,1972.0,-0.93,8.36,23.88,34.24,80966.0,26165.0,339053.0,-15.65,59.0,X-LC,4.00,2.0,0.32,2.43,17.34,X40,NTT,IT
1,ABB,7934.0,-0.68,-3.70,51.15,45.57,128873.0,-9667.0,251952.0,-38.07,57.0,H-MC,4.55,120.0,-0.08,1.80,7.21,AR,NTT,ELECTRICAL
27,GLAXO,3466.2,0.33,3.83,34.01,39.15,70373.0,7636.0,206920.0,-16.56,54.0,X-MC,10.70,60.0,0.11,1.48,33.94,X40,ATH,PHARMA


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.28
1,25,44.89
2,50,76.66


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.91
MC    30.98
LC    24.08
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.71
X40      22.96
X40N     13.09
XR        9.73
XY25      9.08
AR        9.06
OX40N     7.58
SR        0.99
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.50
H-SC    23.78
X-LC    20.65
M-SC    11.67
X-SC     8.05
H-MC     4.82
L-SC     1.41
M-MC     1.32
M-LC     1.29
H-LC     1.13
L-LC     1.01
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.40,-7.28,41.77
IT,12.99,-15.81,74.04
FINANCE,9.48,-20.17,72.43
MISC,7.26,-27.83,80.12
ELECTRICAL,6.02,-11.35,52.14
PAINTS,5.85,-12.83,29.59
INSURANCE,4.77,-1.84,36.69
PHARMA,4.12,-2.04,34.17
AUTO,2.81,-33.50,79.79


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3188391.0,21
AR,1306102.0,10
XR,1301105.0,13
X40,1012707.0,14
X40N,856012.0,9
OX40N,730222.0,10
XY25,372151.0,6
SR,281819.0,2
MH,77100.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3655101.0,25
M-SC,1426671.0,15
X-MC,1401516.0,16
X-LC,839445.0,11
X-SC,792770.0,8
H-MC,405308.0,3
L-SC,263908.0,3
M-LC,111771.0,1
H-LC,75538.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1279015.0      6
           AR         893593.0      5
M-SC       XY24       822884.0      6
H-SC       XR         782571.0      7
X-MC       X40        489510.0      7
           XY24       401918.0      3
H-SC       OX40N      341003.0      4
X-LC       X40        335060.0      5
X-MC       X40N       332914.0      4
X-SC       X40N       307532.0      3
M-SC       OX40N      303953.0      5
X-SC       XY24       297101.0      3
H-SC       SR         281819.0      2
X-LC       X40N       215566.0      2
H-MC       AR         210810.0      2
           XY24       194498.0      1
X-LC       XY24       192975.0      2
X-SC       X40        188137.0      2
L-SC       XR         178642.0      2
X-MC       XY25       177174.0      2
M-SC       XR         173673.0      2
           AR         126161.0      2
M-LC       XR         111771.0      1
X-LC       XY25        95844.0      2
L-SC       OX40N       85266.0      1
H-SC       MH          77100.0      1
H-LC       AR          75538.0      1
M-MC       XY25        60783.0      1
L-MC       XR          54448.0      1
L-LC       XY25        38350.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 59.0 seconds
